In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family']='SimHei' #顯示中文

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('input/train.csv', encoding = "utf-8", dtype = {'type': np.int32})
test = pd.read_csv('input/test.csv', encoding = "utf-8")

In [4]:
#把示範用的 type 4, 資料去除, 以免干擾建模
train = train[train['type']!=4]

In [5]:
from sklearn.model_selection import train_test_split

X = train[['花瓣寬度','花瓣長度','花萼寬度','花萼長度']]
y = train['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=100)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [7]:
from sklearn.svm import SVC
from sklearn import metrics

svc = SVC(C=1.0, kernel="rbf", probability=True)
svc.fit(X_train_std, y_train)

print(metrics.classification_report(y_test, svc.predict(X_test_std)))
print(metrics.confusion_matrix(y_test, svc.predict(X_test_std)))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       1.00      0.90      0.95        10
          3       0.92      1.00      0.96        12

avg / total       0.97      0.97      0.97        36

[[14  0  0]
 [ 0  9  1]
 [ 0  0 12]]


## Grid Search
<p>scikit-learn example: http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html</p>
<p>scikit-learn    官網: http://scikit-learn.org/stable/</p>
<p>         中文教學說明: https://www.gitbook.com/book/htygithub/machine-learning-python/details </p>


In [8]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

for score in scores:
    print("# Tuning hyper-parameters for < %s > -------------------------" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print("mean_test_score (+/- std_test_score) for { paramaters..}")
    print("")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for < precision > -------------------------

Best parameters set found on development set:

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:
mean_test_score (+/- std_test_score) for { paramaters..}

0.119 (+/-0.011) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.119 (+/-0.011) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.940 (+/-0.105) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.119 (+/-0.011) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.947 (+/-0.119) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.940 (+/-0.105) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.927 (+/-0.129) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.947 (+/-0.119) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.934 (+/-0.127) for {'C': 1, 'kernel': 'linear'}
0.937 (+/-0.105) for {'C': 10, 'kernel': 'linear'}
0.921 (+/-0.134) for {'C': 100, 'kernel': 'linear'}
0.921 (+/-0.134) for {'C': 1000, 'kernel': 'linear'}

Detailed classificat

## Classifier comparison
<p>ref. https://machine-learning-python.kspax.io/Classification/ex4_Classifier_comparison.html</p>

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
         "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Ana.",
         "Quadratic Discriminant Ana."]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, probability=True),
    SVC(kernel='rbf',C=100, gamma=0.001, probability=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
X = train[['花瓣寬度','花瓣長度','花萼寬度','花萼長度']]
y = train['type']

sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=100)

test_std = sc.transform(test[['花瓣寬度','花瓣長度','花萼寬度','花萼長度']])

# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%r score: %0.04f" % (name, score))
    
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(clf.predict(test_std))
    print("--------------------------------------------------------------")
    

'Nearest Neighbors' score: 0.9722
             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       1.00      0.90      0.95        10
          3       0.92      1.00      0.96        12

avg / total       0.97      0.97      0.97        36

[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 2 3 3 3]
--------------------------------------------------------------
'Linear SVM' score: 0.8889
             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       0.80      0.80      0.80        10
          3       0.83      0.83      0.83        12

avg / total       0.89      0.89      0.89        36

[1 1 1 1 1 1 1 1 1 1 3 2 3 2 2 2 3 2 2 2 3 2 3 3 3 3 2 3 3 3]
--------------------------------------------------------------
'RBF SVM' score: 0.9722
             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       1.00

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

clf1 = KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf2 = RandomForestClassifier(n_estimators=500, criterion='gini', max_features='auto', oob_score=True)
clf3 = GaussianNB()
clf4 = SVC(kernel='rbf',C=100, gamma=0.001, probability=True)
clf5 = AdaBoostClassifier()
clf6 = LinearDiscriminantAnalysis()
clf7 = QuadraticDiscriminantAnalysis()
clf8 = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4)
meta_clf = LogisticRegression()
stacking_clf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8], meta_classifier=meta_clf)

clf1.fit(X_train_std, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train_std, y_train)
clf4.fit(X_train_std, y_train)
clf5.fit(X_train_std, y_train)
clf6.fit(X_train_std, y_train)
clf7.fit(X_train_std, y_train)
clf8.fit(X_train_std, y_train)
stacking_clf.fit(X_train_std, y_train)

print('KNN Score:',clf1.score(X_test_std, y_test))
print('RF Score:',clf2.score(X_test, y_test))
print('GNB Score:',clf3.score(X_test_std, y_test))
print('SVC Score:',clf4.score(X_test_std, y_test))
print('AdaBoost Score:',clf5.score(X_test_std, y_test))
print('L-Discr. Score:',clf6.score(X_test_std, y_test))
print('Q-Discr. Score:',clf7.score(X_test_std, y_test))
print('XGBoost Score:',clf8.score(X_test_std, y_test))
print('Stacking Score:',stacking_clf.score(X_test_std, y_test))
print("--------------------------------------------------------------")
print(stacking_clf.predict(test_std))

KNN Score: 0.944444444444
RF Score: 0.972222222222
GNB Score: 0.972222222222
SVC Score: 0.972222222222
AdaBoost Score: 1.0
L-Discr. Score: 1.0
Q-Discr. Score: 1.0
XGBoost Score: 0.972222222222
Stacking Score: 0.972222222222
--------------------------------------------------------------
[1 1 1 1 1 1 1 1 1 1 2 2 3 2 2 2 2 2 2 2 3 3 3 3 3 3 2 3 3 3]


In [13]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

clf1 = LinearDiscriminantAnalysis()
clf2 = QuadraticDiscriminantAnalysis()
clf3 = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4)
meta_clf = LinearDiscriminantAnalysis()
stacking_clf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=meta_clf)

clf1.fit(X_train_std, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train_std, y_train)
stacking_clf.fit(X_train_std, y_train)

print('L-Discr. Score:',clf1.score(X_test_std, y_test))
print('Q-Discr. Score:',clf2.score(X_test_std, y_test))
print('XGBoost Score:',clf3.score(X_test_std, y_test))
print('Stacking Score:',stacking_clf.score(X_test_std, y_test))
print("--------------------------------------------------------------")
print(stacking_clf.predict(test_std))

L-Discr. Score: 1.0
Q-Discr. Score: 1.0
XGBoost Score: 0.972222222222
Stacking Score: 1.0
--------------------------------------------------------------
[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3]


In [14]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

clf1 = KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf2 = RandomForestClassifier(n_estimators=500, criterion='gini', max_features='auto', oob_score=True)
clf3 = GaussianNB()
clf4 = SVC(kernel='rbf',C=100, gamma=0.001, probability=True)
clf5 = AdaBoostClassifier()
clf6 = LinearDiscriminantAnalysis()
clf7 = QuadraticDiscriminantAnalysis()
clf8 = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4)

eclf = VotingClassifier(estimators=[('knn', clf1), ('rfc', clf2), ('gnb', clf3), ('svc', clf4),
                                    ('Ada', clf5), ('Lda', clf6), ('Qda', clf7), ('XGB', clf8)], 
                        voting='hard', weights=[1, 1, 1, 1, 1, 1, 1, 1])
eclf.fit(X_train_std, y_train)
print(metrics.classification_report(y_test, eclf.predict(X_test_std)))
print("--------------------------------------------------------------")
print(eclf.predict(test_std))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       1.00      0.90      0.95        10
          3       0.92      1.00      0.96        12

avg / total       0.97      0.97      0.97        36

--------------------------------------------------------------
[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 2 3 3 3]


In [15]:
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

clf1 = LinearDiscriminantAnalysis()
clf2 = QuadraticDiscriminantAnalysis()
clf3 = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4)

eclf = VotingClassifier(estimators=[('Lda', clf1), ('Qda', clf2), ('XGB', clf3)], 
                        voting='hard', weights=[1, 1, 1])
eclf.fit(X_train_std, y_train)
print(metrics.classification_report(y_test, eclf.predict(X_test_std)))
print("--------------------------------------------------------------")
print(eclf.predict(test_std))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00        14
          2       1.00      1.00      1.00        10
          3       1.00      1.00      1.00        12

avg / total       1.00      1.00      1.00        36

--------------------------------------------------------------
[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3]
